In [2]:
# === Scenario Generator with Realistic Stochastic Wind and 10-Min Resolution (20 Scenarios) ===
import numpy as np
import pandas as pd
import os
import json
import requests
import pypsa
from datetime import datetime

# === CONFIG ===
SCENARIO_DURATION_MONTHS = 6
OUTPUT_DIR = "ml_scenarios_10min_6months"
RESOLUTION = "10min"
FREQ_MINUTES = 10
STEPS_PER_HOUR = int(60 / FREQ_MINUTES)

os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Fixed Scenario Dates: Jan 1st and Jul 1st from 2015 to 2024 ===
SCENARIO_DATES = [
    '2015-01-01', '2015-07-01',
    '2016-01-01', '2016-07-01',
    '2017-01-01', '2017-07-01',
    '2018-01-01', '2018-07-01',
    '2019-01-01', '2019-07-01',
    '2020-01-01', '2020-07-01',
    '2021-01-01', '2021-07-01',
    '2022-01-01', '2022-07-01',
    '2023-01-01', '2023-07-01',
    '2024-01-01', '2024-07-01'
]

# === Real Market Prices ===
def fetch_elspot_prices(start_date, end_date, price_area="DK1"):
    url = "https://api.energidataservice.dk/dataset/Elspotprices"
    params = {
        "start": f"{start_date}T00:00", "end": f"{end_date}T23:59",
        "filter": json.dumps({"PriceArea": [price_area]}),
        "columns": "HourDK,SpotPriceDKK", "sort": "HourDK ASC", "timezone": "dk"
    }
    response = requests.get(url, params=params)
    records = response.json().get("records", [])
    df = pd.DataFrame(records)
    df["HourDK"] = pd.to_datetime(df["HourDK"])
    df = df.drop_duplicates("HourDK").set_index("HourDK").sort_index()
    df = df.resample(RESOLUTION).interpolate()
    return df.rename(columns={"SpotPriceDKK": "market_price"})

# === Real Load Data ===
def fetch_real_load_data(start_date, end_date, grid_area_code="031"):
    url = "https://api.energidataservice.dk/dataset/ConsumptionPerGridarea"
    params = {
        'start': f'{start_date}T00:00',
        'end': f'{end_date}T23:59',
        'filter': json.dumps({"GridAreaCode": [grid_area_code]}),
        'sort': 'HourUTC ASC',
        'timezone': 'utc',
        'limit': 100000
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print("❌ Failed to fetch load data")
        print("Status Code:", response.status_code)
        print("Response Text:", response.text)
        raise RuntimeError("Energinet API request failed for load data.")

    try:
        data = response.json().get("records", [])
    except json.JSONDecodeError as e:
        print("❌ JSON decode error:", e)
        print("Response Text:", response.text)
        raise RuntimeError("Failed to decode JSON from Energinet load API.")

    if not data:
        raise ValueError("⚠️ No load data returned from API — check date range and GridAreaCode.")

    df = pd.DataFrame(data)
    df["HourUTC"] = pd.to_datetime(df["HourUTC"])
    df = df[df["GridAreaCode"] == grid_area_code].drop_duplicates("HourUTC").set_index("HourUTC").sort_index()
    df = df.resample(RESOLUTION).interpolate()
    return df.rename(columns={"Consumption": "real_load"})[["real_load"]]


print("📈 Fetching real market price data (2015-2024)...")
real_prices = fetch_elspot_prices("2015-01-01", "2024-12-31")
print("⚡ Fetching real load data (2015-2024)...")
real_loads = fetch_real_load_data("2015-01-01", "2024-12-31")

# === Generation Profiles ===
def realistic_wind_power_curve(speed):
    cut_in, rated, cut_out = 3, 12, 25
    output = np.zeros_like(speed)
    output[(speed >= cut_in) & (speed <= rated)] = (
        (speed[(speed >= cut_in) & (speed <= rated)] - cut_in) / (rated - cut_in)
    )
    output[(speed > rated) & (speed < cut_out)] = 1
    return np.clip(output, 0, 1)

def generate_profiles(length, wind_mean, solar_scale, hydro_shift, timestamps):
    base_wind = np.random.normal(wind_mean, 5, length)
    gust_effect = 0.5 * np.sin(np.linspace(0, 30 * np.pi, length))
    turbulence = np.random.normal(0, 1, length)
    wind_speed = np.maximum(base_wind + gust_effect + turbulence, 0)
    wind_pu = realistic_wind_power_curve(wind_speed)
    wind_pu += np.random.normal(0, 0.05, size=wind_pu.shape)
    wind_pu = np.clip(wind_pu, 0, 1.5)

    solar_profile = np.clip(np.sin(np.linspace(0, 20 * np.pi, length)) * solar_scale, 0, 1)
    seasonal_scaler = np.clip(np.cos((timestamps.dayofyear - 172) * 2 * np.pi / 365), 0, 1)
    solar_profile *= seasonal_scaler
    solar_profile *= np.random.normal(1.0, 0.1, size=length)

    hydro_profile = 0.5 + 0.3 * np.sin(np.linspace(0, 5 * np.pi, length)) + hydro_shift
    hydro_profile += np.random.normal(0, 0.05, length)
    hydro_profile = np.clip(hydro_profile, 0.1, 1.0)

    return wind_pu, solar_profile, hydro_profile

# === Network Build ===
def build_network(timestamps, wind_pu, solar_pu, hydro_pu, load_profile, carbon_tax, wind_capacity):
    network = pypsa.Network()
    network.set_snapshots(timestamps)

    for bus in ["Wind", "Solar", "Hydro", "Grid"]:
        network.add("Bus", bus, carrier="electricity")

    network.add("Generator", "Wind", bus="Wind", p_nom=wind_capacity, p_max_pu=wind_pu,
                capital_cost=1000000, marginal_cost=2 + carbon_tax)
    network.add("Generator", "Solar", bus="Solar", p_nom=1000, p_max_pu=solar_pu,
                capital_cost=600000, marginal_cost=1 + carbon_tax)
    network.add("Generator", "Hydro", bus="Hydro", p_nom=1000, p_max_pu=hydro_pu,
                capital_cost=1200000, marginal_cost=0.5)

    network.add("Load", "Grid_Load", bus="Grid", p_set=load_profile)
    network.add("Generator", "Slack", bus="Grid", p_nom=1e6, p_min_pu=-1, p_max_pu=1, marginal_cost=1e6)

    for src in ["Wind", "Solar", "Hydro"]:
        network.add("Line", f"{src}_to_Grid", bus0=src, bus1="Grid", s_nom=2000, x=0.01, r=0.01, capital_cost=0)

    for comp in ["buses", "generators", "loads", "lines"]:
        getattr(network, comp)["carrier"] = "electricity"

    return network

# === Run Simulation ===
def run_simulation(scenario_id, scenario_start_date):
    steps = SCENARIO_DURATION_MONTHS * 30 * 24 * STEPS_PER_HOUR
    timestamps = pd.date_range(start=scenario_start_date, periods=steps, freq=RESOLUTION)

    np.random.seed(int(scenario_id.split("_")[-1]))
    wind_mean = np.random.uniform(6, 12)
    solar_scale = np.random.uniform(0.8, 1.3)
    hydro_shift = np.random.uniform(-0.3, 0.3)
    carbon_tax = np.random.choice([0, 20, 50])
    wind_capacity = 100 * 15  # 100 turbines × 15 MW

    wind_pu, solar_pu, hydro_pu = generate_profiles(len(timestamps), wind_mean, solar_scale, hydro_shift, timestamps)

    load_df = real_loads.reindex(timestamps, method="nearest")
    load_profile = load_df["real_load"].values

    network = build_network(timestamps, wind_pu, solar_pu, hydro_pu, load_profile, carbon_tax, wind_capacity)
    network.optimize(solver_name="glpk")

    prices = real_prices.reindex(timestamps, method="nearest")["market_price"].values

    df = pd.DataFrame({
        "timestamp": timestamps,
        "wind": network.generators_t.p["Wind"].values,
        "solar": network.generators_t.p["Solar"].values,
        "hydro": network.generators_t.p["Hydro"].values,
        "load": load_profile,
        "market_price": prices,
        "scenario": scenario_id
    })

    df["revenue"] = (df["wind"] + df["solar"] + df["hydro"]) * df["market_price"]
    df["npv"] = df["revenue"].sum() - (wind_capacity * 1000 + 600000 + 1200000)
    df["risk"] = df["revenue"].std() / df["revenue"].mean()
    df["profit_label"] = (df["npv"] > 0).astype(int)

    df.to_csv(f"{OUTPUT_DIR}/{scenario_id}.csv", index=False)
    return df

# === MAIN LOOP ===
all_scenarios = []
for i, start_date_str in enumerate(SCENARIO_DATES):
    scenario_id = f"scenario_{i:03d}"
    print(f"Running {scenario_id} starting at {start_date_str}...")
    df = run_simulation(scenario_id, start_date_str)
    all_scenarios.append(df)

final_df = pd.concat(all_scenarios)
final_df.to_csv(f"{OUTPUT_DIR}/full_training_data_real_price_wind_load.csv", index=False)
print("\n✅ All 20 scenarios saved with 10-minute resolution, real load & price, and realistic wind farm profile.")


📈 Fetching real market price data (2015-2024)...
⚡ Fetching real load data (2015-2024)...
❌ Failed to fetch load data
Status Code: 400
Response Text: Invalid column GridAreaCode


RuntimeError: Energinet API request failed for load data.